In [ ]:
import pandas as pd
import numpy as np
import seaborn  as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,RobustScaler , OrdinalEncoder ,LabelEncoder
from sklearn.metrics import mean_squared_error , mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from google.colab import files

In [ ]:
files.upload()

Saving airbnb_listing_train.csv to airbnb_listing_train.csv
Saving airbnb_listing_validate.csv to airbnb_listing_validate.csv
Saving sample_submission.csv to sample_submission.csv


{'airbnb_listing_train.csv': b'id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,price\n11602914,Luxury home 100m2+garden in center!,3123809,Marjolein,,De Pijp - Rivierenbuurt,52.35368,4.90177,Entire home/apt,3,13,15-02-2020,0.27,1,0,220\n13289321,East is where your home is!,10259430,Ana,,Oud-Oost,52.357279999999996,4.92052,Entire home/apt,4,14,19-06-2019,0.29,1,0,110\n40779315,Amsterdam Central - LUXURY house in BUSY STREET,224969266,Mark,,Centrum-West,52.375820000000004,4.8985,Entire home/apt,2,9,16-03-2020,1.65,1,7,100\n7820311,great appartment down town,693472,Ayden,,Westerpark,52.37792,4.873,Entire home/apt,3,42,17-02-2020,0.72,1,0,130\n27346603,Private room with canal view in peaceful area,41888346,Alissa,,Westerpark,52.38051,4.87493,Private room,2,89,26-02-2020,4.02,1,24,90\n6119654,Central&Cosy place @ Rijksmuseum!,4056223,Sophie,

In [70]:
df_train=pd.read_csv('airbnb_listing_train.csv')
df_test=pd.read_csv('airbnb_listing_validate.csv')
df_ytest=pd.read_csv('sample_submission.csv')

In [ ]:
df_train.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,price
0,11602914,Luxury home 100m2+garden in center!,3123809,Marjolein,NaN,De Pijp - Rivierenbuurt,52.35368,4.90177,Entire home/apt,3,13,15-02-2020,0.27,1,0,220
1,13289321,East is where your home is!,10259430,Ana,NaN,Oud-Oost,52.35728,4.92052,Entire home/apt,4,14,19-06-2019,0.29,1,0,110
2,40779315,Amsterdam Central - LUXURY house in BUSY STREET,224969266,Mark,NaN,Centrum-West,52.37582,4.89850,Entire home/apt,2,9,16-03-2020,1.65,1,7,100
3,7820311,great appartment down town,693472,Ayden,NaN,Westerpark,52.37792,4.87300,Entire home/apt,3,42,17-02-2020,0.72,1,0,130
4,27346603,Private room with canal view in peaceful area,41888346,Alissa,NaN,Westerpark,52.38051,4.87493,Private room,2,89,26-02-2020,4.02,1,24,90


In [6]:
df_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12901 entries, 0 to 12900
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              12901 non-null  int64  
 1   name                            12879 non-null  object 
 2   host_id                         12901 non-null  int64  
 3   host_name                       12893 non-null  object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   12901 non-null  object 
 6   latitude                        12901 non-null  float64
 7   longitude                       12901 non-null  float64
 8   room_type                       12901 non-null  object 
 9   minimum_nights                  12901 non-null  int64  
 10  number_of_reviews               12901 non-null  int64  
 11  last_review                     11305 non-null  object 
 12  reviews_per_month               

In [7]:
df_train.describe()

,id,host_id,neighbourhood_group,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,price
count,1.290100e+04,1.290100e+04,0.0,12901.000000,12901.000000,12901.000000,12901.000000,11305.000000,12901.000000,12901.000000,12901.000000
mean,1.984923e+07,6.321700e+07,NaN,52.365313,4.889744,3.542516,24.803194,0.748501,2.136501,61.392295,166.959538
std,1.216241e+07,8.010814e+07,NaN,0.016418,0.036016,15.154077,53.406134,1.260791,6.001838,107.988329,234.794630
min,2.016800e+04,3.592000e+03,NaN,52.289270,4.755720,1.000000,0.000000,0.010000,1.000000,0.000000,6.000000
25%,9.869642e+06,8.948269e+06,NaN,52.355190,4.863950,2.000000,2.000000,0.180000,1.000000,0.000000,99.000000
50%,1.874939e+07,2.711601e+07,NaN,52.364570,4.887130,2.000000,9.000000,0.380000,1.000000,0.000000,135.000000
75%,2.914232e+07,8.337686e+07,NaN,52.375110,4.909310,3.000000,24.000000,0.770000,1.000000,87.000000,190.000000
max,4.370900e+07,3.490175e+08,NaN,52.425090,5.018400,1001.000000,843.000000,50.000000,78.000000,365.000000,9000.000000


In [8]:
df_train.isna().sum()

id                                    0
name                                 22
host_id                               0
host_name                             8
neighbourhood_group               12901
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
minimum_nights                        0
number_of_reviews                     0
last_review                        1596
reviews_per_month                  1596
calculated_host_listings_count        0
availability_365                      0
price                                 0
dtype: int64

Dropping name , host_name  as we have id and host_id
, Dropping neighbourhood_group since it doesnt have any entries

---



In [71]:
df_train.drop(['name','host_name','neighbourhood_group','last_review'],1,inplace=True)

<ipython-input-71-16bed142e0a0>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_train.drop(['name','host_name','neighbourhood_group','last_review'],1,inplace=True)


In [13]:
df_train.isna().sum()

id                                   0
host_id                              0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
minimum_nights                       0
number_of_reviews                    0
reviews_per_month                 1596
calculated_host_listings_count       0
availability_365                     0
price                                0
dtype: int64

In [72]:
df_train['reviews_per_month']=df_train['reviews_per_month'].fillna(df_train['reviews_per_month'].median())

In [26]:
df_train.isna().sum()

id                                0
host_id                           0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
price                             0
dtype: int64

In [31]:
df_train.sample(5)

,id,host_id,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,price
3803,41848407,330804538,De Pijp - Rivierenbuurt,52.35728,4.90106,Private room,3,0,0.38,1,90,120
11457,956611,1477248,Centrum-West,52.37631,4.88982,Entire home/apt,7,51,0.58,9,0,130
12135,4444190,20125546,De Baarsjes - Oud-West,52.35971,4.85377,Entire home/apt,4,2,0.03,1,0,150
9221,20115491,19722616,Oud-Oost,52.35880,4.92555,Entire home/apt,3,10,0.29,1,0,100
2518,31146092,232797683,Westerpark,52.37900,4.87487,Entire home/apt,3,5,0.42,1,0,100


In [73]:
df_train['room_type'].value_counts()

Entire home/apt    10064
Private room        2622
Hotel room           174
Shared room           41
Name: room_type, dtype: int64

In [74]:
df_train=df_train.join(pd.get_dummies(df_train['room_type']))

In [75]:
df_train.drop('room_type',1,inplace=True)

<ipython-input-75-e383449f8d8c>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_train.drop('room_type',1,inplace=True)


In [76]:
df_train['neighbourhood'] = df_train['neighbourhood'].astype('category')
df_train['neighbourhood'] = df_train['neighbourhood'].cat.codes


In [50]:
df_train.dtypes

id                                  int64
host_id                             int64
neighbourhood                        int8
latitude                          float64
longitude                         float64
minimum_nights                      int64
number_of_reviews                   int64
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
price                               int64
Entire home/apt                     uint8
Hotel room                          uint8
Private room                        uint8
Shared room                         uint8
dtype: object

In [55]:
df_test.sample(5)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
1996,18014100,Large family home with garden in East of Amste...,7011756,Marike,NaN,Oud-Oost,52.35964,4.92513,Entire home/apt,2,5,28-07-2018,0.13,1,0
1127,34738226,Fantastic apt in bustling center of A'dam + bath,39113987,BnB-Beheer,NaN,Centrum-Oost,52.36187,4.88459,Entire home/apt,2,10,31-07-2019,0.80,11,0
2054,7448305,LaCorteSconta B&B - Amsterdam Floating-Villa,39009584,Yara,NaN,IJburg - Zeeburgereiland,52.36415,4.98588,Private room,1,57,09-02-2020,0.96,3,218
1685,12540879,Cosy apartment Amsterdam west,14693799,Marion,NaN,Westerpark,52.38378,4.87679,Entire home/apt,3,8,27-08-2017,0.16,1,0
5038,8675811,3 ROOMappartement @ Buikslotermeerplein North,11714074,Leen,NaN,Noord-Oost,52.39772,4.94237,Entire home/apt,2,0,NaN,NaN,1,0


In [77]:
df_test=df_test.drop(['name','host_name','neighbourhood_group','last_review'],1)

<ipython-input-77-cf3ebb973fcf>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_test=df_test.drop(['name','host_name','neighbourhood_group','last_review'],1)


In [78]:
df_test['reviews_per_month']=df_test['reviews_per_month'].fillna(df_test['reviews_per_month'].median())
df_test=df_test.join(pd.get_dummies(df_test['room_type']))
df_test.drop('room_type',1,inplace=True)
df_test['neighbourhood'] = df_test['neighbourhood'].astype('category')
df_test['neighbourhood'] = df_test['neighbourhood'].cat.codes


<ipython-input-78-b478f4cc644a>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_test.drop('room_type',1,inplace=True)


In [80]:
df_test.sample(5)

,id,host_id,neighbourhood,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,Entire home/apt,Hotel room,Private room,Shared room
2288,20736983,126871692,8,52.35285,4.89082,3,1,0.03,1,0,1,0,0,0
1331,17541451,28148554,8,52.34852,4.90648,2,3,0.08,1,0,1,0,0,0
4589,18193212,125556535,9,52.30526,4.97721,30,45,1.21,1,364,1,0,0,0
216,11843850,9462022,5,52.36655,4.89039,3,65,1.28,3,346,1,0,0,0
3371,5427607,28154037,8,52.34624,4.89222,2,28,0.47,1,25,1,0,0,0


In [79]:
df_train.shape,df_test.shape

((12901, 15), (6451, 14))

model

In [53]:
x=df_train.drop('price',1)
y=df_train[['price']]


<ipython-input-53-35207c534f76>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x=df_train.drop('price',1)


In [81]:
model=LinearRegression()
model.fit(x,y)
pred=model.predict(df_test)

In [98]:
pred[:10]

array([[243.97190528],
       [181.57687857],
       [155.12748274],
       [123.89196667],
       [151.17497346],
       [192.87377377],
       [192.26226481],
       [155.8243729 ],
       [233.31218967],
       [144.84255217]])